<a href="https://colab.research.google.com/github/ssubin29/DaconAlone/blob/main/2021_ForecastingPriceContest/LoadAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*이탤릭체 텍스트*

In [ ]:
import pandas as pd
import urllib.request
import json
import holidays
import datetime
import numpy as np
from tqdm import tqdm
from datetime import timedelta, date
from dateutil.relativedelta import relativedelta
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import folium
from glob import glob
import multiprocessing
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_IN = '/content/drive/Shareddrives/2021_ForecastingPriceContest/dataset'
data = pd.read_csv(DATA_IN + '/public_data/train.csv')
empty_df = pd.DataFrame(data.loc[[2],:])
empty_df

FileNotFoundError: ignored

In [ ]:
days = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일']
def loadAPI(date):  
    global main_df  
    try:
        url_date = date.strftime("%Y%m%d")
        url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

        response = urllib.request.urlopen(url+url_date).read()
        response = json.loads(response)

        data = pd.DataFrame(response['data'])
        preprocessed_data = preprocessing(data)
        preprocessed_data['date'] = date
        preprocessed_data['요일'] = days[date.weekday()]

        main_df = main_df.append(preprocessed_data)
    except:
        empty_df['date'] = date 
        empty_df['요일'] = days[date.weekday()]
        main_df = main_df.append(empty_df)

In [ ]:
# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '건고추','마늘',
        '대파', '얼갈이배추', '양배추', '깻잎',
        '시금치', '미나리', '당근',
        '파프리카', '새송이', '팽이버섯', '토마토',
    ]
    
    unique_kind = [
        '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
    ]
    
    train_dict = {
        'date':[]
    }
    
    for sub in unique_pum:
        train_dict[f'{sub}_거래량(kg)'] = []
        train_dict[f'{sub}_가격(원/kg)'] = []
        
    for sub in unique_kind:
        train_dict[f'{sub}_거래량(kg)'] = []
        train_dict[f'{sub}_가격(원/kg)'] = []
        
    tsalet_sample = tsalet_file
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        train_dict['date'].append(day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)'].append(0)
                train_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                train_dict[f'{sub}_가격(원/kg)'].append(mean_price)
        
        for sub in unique_kind:
            # 날짜별, 품종별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['KIND_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)'].append(0)
                train_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = round(tot_amt/(tot_qty+1e-20))
                tot_qty = round(tot_qty, 1)
                train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                train_dict[f'{sub}_가격(원/kg)'].append(mean_price)

    return pd.DataFrame(train_dict)     

In [ ]:
main_df = pd.DataFrame(columns=['date', '요일', '배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '건고추_거래량(kg)', '건고추_가격(원/kg)',
       '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)', '대파_가격(원/kg)',
       '얼갈이배추_거래량(kg)', '얼갈이배추_가격(원/kg)', '양배추_거래량(kg)', '양배추_가격(원/kg)',
       '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)', '시금치_가격(원/kg)',
       '미나리_거래량(kg)', '미나리_가격(원/kg)', '당근_거래량(kg)', '당근_가격(원/kg)',
       '파프리카_거래량(kg)', '파프리카_가격(원/kg)', '새송이_거래량(kg)', '새송이_가격(원/kg)',
       '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)',
       '청상추_거래량(kg)', '청상추_가격(원/kg)', '백다다기_거래량(kg)', '백다다기_가격(원/kg)',
       '애호박_거래량(kg)', '애호박_가격(원/kg)', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',
       '샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)'])

In [ ]:
data_start = date(2020,10,1)
data_end = date(2021,11,5)
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

for n in range(14):
    start = data_start + relativedelta(months=n)
    end = start + relativedelta(months=1) - timedelta(1)
    for single_date in daterange(start, end):
        print(single_date.strftime("%Y-%m-%d"))  
        loadAPI(single_date)
    main_df.to_csv(f'/content/drive/Shareddrives/2021_ForecastingPriceContest/output/{n}th_data.csv' ,
                   index=False, encoding='utf-8-sig')

2022-08-17
2022-08-18
2022-08-19
2022-08-20
2022-08-21
2022-08-22
2022-08-23
2022-08-24
2022-08-25


In [ ]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2020, 9, 29)
end_date = date(2021, 11, 5)
for single_date in daterange(start_date, end_date):
    print(single_date.strftime("%Y-%m-%d"))
    loadAPI(single_date)
main_df.to_csv('data.csv')

main_df

# 데이터 병합

In [ ]:
DATA_IN = "/content/drive/Shareddrives/2021_ForecastingPriceContest/output"

In [ ]:
merge_df = pd.DataFrame(columns=['date', '요일', '배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '건고추_거래량(kg)', '건고추_가격(원/kg)',
       '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)', '대파_가격(원/kg)',
       '얼갈이배추_거래량(kg)', '얼갈이배추_가격(원/kg)', '양배추_거래량(kg)', '양배추_가격(원/kg)',
       '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)', '시금치_가격(원/kg)',
       '미나리_거래량(kg)', '미나리_가격(원/kg)', '당근_거래량(kg)', '당근_가격(원/kg)',
       '파프리카_거래량(kg)', '파프리카_가격(원/kg)', '새송이_거래량(kg)', '새송이_가격(원/kg)',
       '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)',
       '청상추_거래량(kg)', '청상추_가격(원/kg)', '백다다기_거래량(kg)', '백다다기_가격(원/kg)',
       '애호박_거래량(kg)', '애호박_가격(원/kg)', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',
       '샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)'])

In [ ]:
import os
for dirname, _, filenames in os.walk(DATA_IN):
    for filename in filenames:
        print(filename)
        merge_df = pd.concat([merge_df,pd.read_csv(os.path.join(dirname, filename))])

0.csv
1.csv
2.csv
3.csv
4.csv
5.csv


In [ ]:
merge_df = merge_df.sort_values(by=['date'])

In [ ]:
#mask = (merge_df['date'] >= '2019-10-1') & (merge_df['date'] <= '2021-11-05')
mask = merge_df['date'] <= '2021-11-05'
filtered_df = merge_df.loc[mask]
filtered_df.tail()

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
119,2021-11-01,월요일,1754622.65,612.637059,2552118.1,440.219583,2078178.75,858.803649,1708.7,10578.912624,102888.8,6413.702823,698053.3,1312.295918,316754.5,1362.837096,560305.2,489.903553,103828.5,8443.715685,183042.20,3692.503931,67042.2,5010.249619,518826.0,854.891862,193746.9,3133.891907,181826.5,2991.918087,116035.5,2852.920942,250732.0,3212.498405,49776.0,5718.0,400605.4,2154.0,445453.8,2601.0,11186.0,6119.0,118166.9,10524.0
120,2021-11-02,화요일,1770020.10,608.982626,2418927.8,400.107347,1705887.20,867.063870,1510.2,11978.239968,157062.8,6286.677571,631986.7,1283.237336,274672.5,1255.009864,659218.9,389.801322,70855.2,7279.044149,178767.19,3363.987452,69876.7,3770.678610,453537.0,794.008804,135876.2,2765.279799,166897.8,2961.459270,113226.5,2756.150565,235696.9,2963.519631,41930.0,4542.0,242966.0,2101.0,277731.8,2262.0,8196.0,6383.0,105298.1,10522.0
121,2021-11-03,수요일,1780275.50,656.531144,2301971.1,420.270478,1600708.70,886.183773,2416.1,9445.205496,110984.1,6869.695416,645887.4,1300.943031,271162.0,1223.001958,707492.0,372.988365,73005.1,6802.782258,185290.33,3166.505268,68231.8,4728.753455,335896.0,842.797488,193266.8,2763.089046,150014.4,2956.650108,118359.0,2670.403949,200540.0,2906.925601,46118.5,3827.0,231834.0,2391.0,272272.4,2321.0,8646.0,6524.0,98357.1,11200.0
122,2021-11-04,목요일,1894155.00,685.867743,2268486.2,437.078125,1680691.90,850.640721,1777.3,11156.678670,130839.6,6361.542110,637456.7,1288.021924,269344.5,1190.914851,631764.9,388.894704,69478.2,6579.245015,187761.14,3037.395720,39965.1,5274.350446,281872.0,795.826847,161195.3,2381.497364,137826.0,2929.172783,106566.0,2570.288535,160121.0,2973.970310,43389.8,3374.0,209968.0,2901.0,211932.6,2576.0,6933.0,6529.0,91138.5,10774.0
123,2021-11-05,금요일,1920218.60,844.246272,2249217.5,460.511051,1868628.80,887.464273,1125.0,11807.409778,113544.8,6452.641090,620728.2,1316.291841,239403.5,1236.517870,622137.3,412.758066,72598.0,7161.056544,172894.70,3106.674334,67427.0,4748.139425,309681.0,818.800427,196365.4,2430.705572,152258.4,2908.154486,109922.0,2523.417633,231147.0,2944.130359,44375.5,3161.0,231385.0,2963.0,278731.0,2831.0,7885.0,6174.0,89487.9,10840.0


In [ ]:
filtered_df.to_csv(f'/content/drive/Shareddrives/2021_ForecastingPriceContest/output/merged_data.csv' ,
                   index=False, encoding='utf-8-sig')

### + 휴일 변수 추가

In [ ]:
# 한국 휴일 객체 생성 
kr_holidays = holidays.KR()

# generate holiday table
filtered_df['holiday'] = filtered_df.date.apply(lambda x: 1 if x in kr_holidays else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
filtered_df

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),holiday
0,2016-01-01,금요일,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,2016-01-02,토요일,80860.00,329.000000,80272.0,360.000000,122787.50,1281.000000,3.0,11000.000000,15019.0,5475.000000,92334.0,1704.000000,6359.0,1331.000000,40028.0,348.000000,4374.9,13242.000000,16550.50,2339.000000,10528.0,1729.000000,13885.0,804.000000,3853.0,3703.000000,15797.0,2576.000000,14634.0,1474.000000,30950.0,1621.000000,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0,0
2,2016-01-03,일요일,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2016-01-04,월요일,1422742.50,478.000000,1699653.7,382.000000,2315079.00,1235.000000,699.0,4464.000000,141638.0,5210.000000,994328.1,1716.000000,262615.5,1212.000000,1074699.1,345.000000,122613.5,9923.000000,427435.10,2153.000000,82113.5,3960.000000,558950.2,794.000000,104930.3,4871.000000,277326.5,2440.000000,159800.0,1750.000000,291057.0,1834.000000,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0,0
4,2016-01-05,화요일,1167241.00,442.000000,1423482.3,422.000000,2092960.10,1213.000000,1112.6,4342.000000,126207.8,5387.000000,787716.0,1715.000000,221850.5,1197.000000,825681.9,350.000000,79055.9,9529.000000,334636.80,2220.000000,80144.0,3333.000000,444353.7,763.000000,100699.5,5129.000000,218465.2,2437.000000,153084.0,1822.000000,194626.5,1833.000000,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2021-11-01,월요일,1754622.65,612.637059,2552118.1,440.219583,2078178.75,858.803649,1708.7,10578.912624,102888.8,6413.702823,698053.3,1312.295918,316754.5,1362.837096,560305.2,489.903553,103828.5,8443.715685,183042.20,3692.503931,67042.2,5010.249619,518826.0,854.891862,193746.9,3133.891907,181826.5,2991.918087,116035.5,2852.920942,250732.0,3212.498405,49776.0,5718.0,400605.4,2154.0,445453.8,2601.0,11186.0,6119.0,118166.9,10524.0,0
120,2021-11-02,화요일,1770020.10,608.982626,2418927.8,400.107347,1705887.20,867.063870,1510.2,11978.239968,157062.8,6286.677571,631986.7,1283.237336,274672.5,1255.009864,659218.9,389.801322,70855.2,7279.044149,178767.19,3363.987452,69876.7,3770.678610,453537.0,794.008804,135876.2,2765.279799,166897.8,2961.459270,113226.5,2756.150565,235696.9,2963.519631,41930.0,4542.0,242966.0,2101.0,277731.8,2262.0,8196.0,6383.0,105298.1,10522.0,0
121,2021-11-03,수요일,1780275.50,656.531144,2301971.1,420.270478,1600708.70,886.183773,2416.1,9445.205496,110984.1,6869.695416,645887.4,1300.943031,271162.0,1223.001958,707492.0,372.988365,73005.1,6802.782258,185290.33,3166.505268,68231.8,4728.753455,335896.0,842.797488,193266.8,2763.089046,150014.4,2956.650108,118359.0,2670.403949,200540.0,2906.925601,46118.5,3827.0,231834.0,2391.0,272272.4,2321.0,8646.0,6524.0,98357.1,11200.0,0
122,2021-11-04,목요일,1894155.00,685.867743,2268486.2,437.078125,1680691.90,850.640721,1777.3,11156.678670,130839.6,6361.542110,637456.7,1288.021924,269344.5,1190.914851,631764.9,388.894704,69478.

In [ ]:
filtered_df.to_csv(f'/content/drive/Shareddrives/2021_ForecastingPriceContest/output/merged_data_plus_holiday.csv' ,
                   index=False, encoding='utf-8-sig')